In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import kinodyn_um.urdfparser as u2c
import numpy as np
import casadi as ca

In [2]:
# ─────────────────────────────────────────────────────────────────────────────
# 1. Load robot model
# ─────────────────────────────────────────────────────────────────────────────
alpha = u2c.URDFparser()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(project_root, 'usage', 'urdf', 'alpha_5_robot.urdf')
alpha.from_file(path_to_urdf)

root = "base_link"
tip  = "alpha_standard_jaws_base_link"
n_joints = alpha.get_n_joints(root, tip)

# ─────────────────────────────────────────────────────────────────────────────
# 2. CasADi symbols
# ─────────────────────────────────────────────────────────────────────────────
q     = ca.MX.sym('q',     n_joints)        # joint positions
q_dot = ca.MX.sym('q_dot', n_joints)        # joint velocities
tau_cmd = ca.MX.sym('tau_cmd', n_joints)   # *controller* torque
dt    = ca.MX.sym('dt')                     # integration step (s)
g     = ca.MX.sym('g')                      # gravity acceleration (e.g. -9.81)

k        = ca.MX.sym('k_',        n_joints, 2)   # tanh sharpness coefficients
viscous  = ca.MX.sym('viscous',   n_joints, 2)   # viscous friction
coulomb  = ca.MX.sym('coulomb',   n_joints, 2)   # Coulomb friction
I_Grotor = ca.MX.sym('I_Grotor',  n_joints, 2)   # rotor inertias
payload_props  = ca.MX.sym("payload_props", 4) # mass, Ixx, Iyy, Izz

lower_joint_limit = ca.MX.sym('lower_limit', n_joints)
upper_joint_limit = ca.MX.sym('upper_limit', n_joints)
EPS_TORQUE = ca.MX.sym('eps_torque', n_joints)

# ─────────────────────────────────────────────────────────────────────────────
# 3. Forward dynamics (from CRBA model produced by urdf2casadi)
# ─────────────────────────────────────────────────────────────────────────────
q_ddot_fun = alpha.get_forward_dynamics_crba(root, tip) # acceleration
tau_hold_fun = alpha.get_active_complaince_tau(root, tip)  # gravity+payload

# ------------------------------------------------------------------
# 4. Holding torque (independent of tau_cmd)
# ------------------------------------------------------------------
tau_hold = tau_hold_fun(q, g, payload_props)

# Decision: if |tau_cmd[i]| < ε  →  use tau_hold[i]
use_hold   = ca.fabs(tau_cmd) < EPS_TORQUE
tau_eff    = ca.if_else(use_hold, tau_hold, tau_cmd)

# ─────────────────────────────────────────────────────────────────────────────
# 6. Joint‑limit saturation with recovery
#    Freeze motion only if it is trying to go farther OUT of bounds.
# ─────────────────────────────────────────────────────────────────────────────
EPS = 1e-1
tau_sat = ca.MX.zeros(n_joints)
v_guard = ca.MX.zeros(n_joints)

for i in range(n_joints):

    above = q[i] >= upper_joint_limit[i] - EPS
    below = q[i] <= lower_joint_limit[i] + EPS

    # Outward velocity?
    vel_out_high = ca.logic_and(above, q_dot[i] > 0)
    vel_out_low  = ca.logic_and(below, q_dot[i] < 0)

    # “Adds energy” if torque points the same way as the outward motion
    adds_energy_high = ca.logic_and(vel_out_high, tau_eff[i] >  0)
    adds_energy_low  = ca.logic_and(vel_out_low,  tau_eff[i] <  0)

    # Also block any *static* outward push (velocity zero but torque outward)
    push_out_high = ca.logic_and(above, tau_eff[i] > 0)
    push_out_low  = ca.logic_and(below, tau_eff[i] < 0)

    violate = ca.logic_or(push_out_high, push_out_low)
    violate = ca.logic_or(violate,       adds_energy_high)
    violate = ca.logic_or(violate,       adds_energy_low)

    tau_sat[i] = ca.if_else(violate, 0, tau_eff[i])

    v_guard[i] = ca.if_else(ca.logic_or(vel_out_high, vel_out_low), 0, q_dot[i])


# ------------------------------------------------------------------
# 5. Forward dynamics with the *selected* torque
# ------------------------------------------------------------------
q_ddot = q_ddot_fun(q, v_guard, tau_sat, g, k, viscous, coulomb, I_Grotor, payload_props)
    
# ------------------------------------------------------------------
# 7. State vector and ODE
#     • q-dot derivative uses v_guard  ⇒ no outward drift
#     • q_dot derivative uses q_ddot   ⇒ real braking dynamics
# ------------------------------------------------------------------
x    = ca.vertcat(q,  q_dot)
xdot = ca.vertcat(v_guard, q_ddot) * dt


# ─────────────────────────────────────────────────────────────────────────────
# 8. Parameter vector (keep one long vector to avoid argument mismatch)
# ─────────────────────────────────────────────────────────────────────────────
p = ca.vertcat(
        dt,
        g,
        ca.reshape(k,       -1, 1),
        ca.reshape(viscous, -1, 1),
        ca.reshape(coulomb, -1, 1),
        ca.reshape(I_Grotor,-1, 1),
        payload_props,
        lower_joint_limit,
        upper_joint_limit,
        EPS_TORQUE)


# ─────────────────────────────────────────────────────────────────────────────
# 7. Integrator (Runge–Kutta over [0,1])
# ─────────────────────────────────────────────────────────────────────────────
dae  = {'x': x, 'ode': xdot, 'p': p, 'u': tau_cmd}
opts = {
    'simplify': True,
    'number_of_finite_elements': 100,
}

intg = ca.integrator('intg', 'rk', dae, 0, 1, opts)

# ─────────────────────────────────────────────────────────────────────────────
# 8. Next‑state function and file export
# ─────────────────────────────────────────────────────────────────────────────
x_next = intg(x0=x, u=tau_cmd, p=p)['xf']

p_sim =ca.vertcat(
    ca.reshape(k,   -1, 1), 
    ca.reshape(viscous,   -1, 1),
    ca.reshape(coulomb,   -1, 1),
    ca.reshape(I_Grotor,  -1, 1)
        )

F_next = ca.Function('Mnext', [x, tau_cmd, dt, g, payload_props, p_sim , lower_joint_limit, upper_joint_limit, EPS_TORQUE], [x_next, tau_sat])

F_next.save('arm.casadi')

# # c , cpp or matlab code generation for forward dynamics
# F_next.generate("F_next_.c")
# os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

# print('✔ arm.casadi saved — ready for simulation')